**车站信息**

StationReference = Name | MayBeInitial | Platforms

|名称 | 是否可初始化? | 站台 |  
 |--|--|--|  
 a = 京沪高速泰安站方向 | 1 | 2, 1 
 b = 京济联络线济南方向 | 1 | 2, 1 
 c = 济郑高速长清方向 | 1 | 1, 2 
 d = 济南西站 | 1 | 1, 5, 7, 6, 11, 10, 12, 13, 16, 17, 2, 3, 4, 8, 9, 14, 15 
 e = 济南西动车所 | 1 | 2, 1 
 f = 京沪高速德州东方向 | 1 | 2, 1 
 g = 石济客专齐河方向 | 1 | 1, 2 
 h = 石济客专济南东方向 | 1 | 2, 1 
 i = 黄河南线路所 | 1 |  
 j = 大漠刘线路所 | 1 |  

**时刻表**  
 
**例子**  
|列车编号|类型|最高时速|动拖布置|flag|车站1|车站2|车站3|   
|--|--|--|--|--|--|--|--|  
|G3472 |COMMUTER |300 |LPPLLPPL |X1 :| b#2#21:40:00#0| e#0#21:45:00#30 |a#0#22:19:00#0|  




**车次相关信息**  
车次信息不变 默认commuter  
依照类型设置最高速度 D->200 G->300....  
车辆细节 先默认 MTTM 后续修改长编或者重连?  
  

**停站相关信息**  
仅三点式 进场车站 停站 离场车站  
始发终到则进行利用抵达时刻等替换  
公式=IF(B2<>$J$2,B2,TEXT(E2-0.5/24,"h:mm:ss"))    
车站 筛选所有车次始发终到，设置字典更改进场离场   

  
**表格信息**  
爬虫以及selenium自动化获取数据   
二者合并为一张工作表并预处理为上图格式  
始发站和终到站更换为进场车站和离场车站  
  

**车站信息**  
由于自动化需要,武汉站分为京广高速场和综合场
基于ORM大小略作修改,仅保留联络线部分  

  

**库函数部分及常规列车信息部分**  
- 导入库函数，处理表格以及替换字符并导出  
- 使用字典建立常规信息的映射关系  

In [1]:
import numpy
import pandas
import datetime
from random import randint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys  # 引用keys包
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time


Excelpath = "data.xlsx"  # Excel时刻表文件路径
TextPath = "train.txt"  # 游戏时刻表文件路径
# 速度和编组以及类型映射关系,0为普速1为动车2为高速
species = {'K': ['120', 'LPPPPPP', 0], 'T': ['140', 'LPPPPPP', 0], 'Z': ['160', 'LPPPPPP', 0],
           'D': ['200', 'LPPL', 1], 'C': ['200', 'LPPL', 1], 'G': ['300', 'LPPLLPPL', 2]}


# 车站-编号,掉向,用时以及运行车辆种类映射关系
# 图片左(0)右(1)侧线路key值相同则掉向,
# 国铁车辆行走左侧,2为数据为左侧股道编号
# [车站编号,车站所在侧(0为左侧),车辆进场股道,车辆离场行走股道,到达中心车站所用时间,行走车辆类型]
station = {
    '京沪高速泰安站方向': ['a', 0, '2', '1', 7, 'G'], '京济联络线济南方向': ['b', 0, '2', '1', 8, 'GD'], '济郑高速长清方向': ['c', 0, '2', '1', 7, 'GD'],
    '济南西站': ['d', -1, '0', '0', 7, 'GD'], '济南西动车所': ['e', -1, '0', '0', 6, 'GDC'],

    '京沪高速德州东方向': ['f', 1, '1', '2', 6, 'GD'], '石济客专齐河方向': ['g', 1, '1', '2', 6, 'GD'], '石济客专济南东方向': ['h', 1, '1', '2', 6, 'GD'],
}
ThisStation = '济南西站'
TotalPlat = 18
# track = {}
# 车型关系--待筛选，默认短编
marshalling = {}

# 始发终到映射关系--待筛选
dst = {}

# 股道到达,图片左右侧线路,,key值不同则掉向
# turnst = {'武九客专武汉方向': 0, '武九客专鄂州方向': 1, '武冈城际黄冈方向': 1}

trainDF = pandas.DataFrame(
    columns=['车次', '类型', '最高时速', '动拖布置', 'flag'])  # 列车整体信息
arriveStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 进场信息
stopStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 停站信息
leaveStDF = pandas.DataFrame(columns=['车站名称', '股道', '到达时间', '停站时间'])  # 离场信息

# 存储字符串形式的最终结果
trainList = []
arriveStList = []
stopStList = []
leaveStList = []


**读取表格及数据处理部分**  
- 1,读取表格并去除空行  
- 2,从车次提取速度等级  
- 3,筛选始发站终到站建立替换字典  
  
  
**车次字符串生成部分**  
1,车次信息部分  
 - 车次号,类型,动拖布置及flag保持默认  
 - 速度等级依照KTZDG等区分映射  

2,停站部分    
 - 停站2(主要车站) 到时及停站时间来自表格部分,股道由由车站大屏生成,终到车则为任意股道  
 - 进场及离场部分 依照游戏先期测试进行平移推算,股道依照国铁车辆靠左行驶安排,  

In [2]:
# 读取文件
sheet0 = pandas.read_excel(io=Excelpath, sheet_name="Sheet1")  # 车次及始发终到和检票口信息

# 去除空行
#sheet0 = sheet0.dropna()
sheet0 = sheet0[~(sheet0["车次"].isnull())]  # 删掉空行

sheet = sheet0
sheet = sheet.reset_index(drop=True)  # 按照车次信息进行连接，合并检票口sheet1
sheet = sheet.drop_duplicates(subset="车次")
# sheet.replace("--:--",numpy.nan())
# sheet=sheet.fillna(value=0)

print(sheet)

trainInfo = sheet["车次"].to_frame()  # 车辆信息
# 统计始发终到车站信息
ts = pandas.concat([sheet0["始发站"].value_counts(),
                   sheet0["终到站"].value_counts()])
totalStation = ts.index

print(set(list(totalStation)))  # 所有始发站和终到站统计


             车次   始发站   终到站        到时        开时     检票口
0    D1601D1604   石家庄   石家庄  11:13:00  11:41:00   检票口2A
1    D1603D1606   石家庄   石家庄  16:41:00  17:05:00   检票口2A
2    D1605D1602   石家庄   石家庄  12:16:00  12:40:00   检票口2A
3    D1607D1608   石家庄   石家庄  19:20:00  19:49:00   检票口2A
4         D1611   石家庄   青岛北  10:09:00  10:14:00   检票口4B
..          ...   ...   ...       ...       ...     ...
386          G7   北京南  上海虹桥  11:23:00  11:26:00   检票口6A
387          G8  上海虹桥   北京南  11:59:00  12:02:00  检票口13A
388        G887   北京南    盐城  09:37:00  09:39:00   检票口6A
389        G888    盐城   北京南  17:51:00  17:54:00  检票口14A
390          G9   北京南  上海虹桥  12:22:00  12:24:00   检票口7A

[391 rows x 6 columns]
{'菏泽东', '黄山北', '深圳北', '郑州东', '青岛北', '石家庄', '平原东', '呼和浩特东', '温州南', '杭州西', '济南西', '江门', '北京南', '上海虹桥', '济南东', '青岛', '运城北', '福州', '盐城', '沈阳北', '宁波', '烟台', '杭州东', '天津西', '曲阜东', '南昌西', '临沂北', '厦门北', '济南', '太原南'}


**始发终到替换**  
存在多路径问题  
- 如从济南站到青岛站,有济青高速和胶济客专,在济南西站的上下行不同   
- 手动替换准确度低

In [3]:
# 手动建立映射关系,别的方法不会

def getArrLeaSt(trainNUm, station):
    speedType = trainNUm[0]
    up_down = int(trainNUm[-1]) % 2
    stcode = ""
    # 多路径车站部分,区分上下行
    if speedType == "D":  # 
        ArrLeaveSt = {
            '青岛北': '京济联络线济南方向',
            '青岛': '京济联络线济南方向',
            '盐城': '京济联络线济南方向',
            '烟台': '京济联络线济南方向',

        }
    else :  # 
        ArrLeaveSt = {
            '青岛北': '石济客专济南东方向',           
            '青岛': '石济客专济南东方向',
            '盐城': '石济客专济南东方向',
            '烟台': '石济客专济南东方向',

        }

    stcode = ArrLeaveSt.get(station)
    if stcode != None:
        print("车次:{0} 车站:{1} ".format(trainNUm, station))
        return stcode

    # 常规单路径车站
    ArrLeaveSt = {
        '济南': '京济联络线济南方向',
        '济南西':'济南西动车所',
        '济南东': '石济客专济南东方向',
        '石家庄': '石济客专齐河方向',
        '太原南': '石济客专齐河方向',
        '运城北': '石济客专齐河方向',
        '郑州东': '济郑高速长清方向',
        '上海虹桥': '京沪高速泰安站方向',
        '呼和浩特东': '石济客专齐河方向',
        '平原东': '石济客专齐河方向',
        '北京南': '京沪高速德州东方向',
        '杭州东': '京沪高速泰安站方向',
        '沈阳北': '京沪高速德州东方向',
        '宁波': '京沪高速泰安站方向',
        '福州': '京沪高速泰安站方向',
        '杭州西': '京沪高速泰安站方向',
        '南昌西': '京沪高速泰安站方向',
        '温州南': '京沪高速泰安站方向',
        '江门': '京沪高速泰安站方向',
        '黄山北': '京沪高速泰安站方向',
        '菏泽东': '京沪高速泰安站方向',
        '天津西': '京沪高速德州东方向',
        '临沂北': '石济客专济南东方向',
        '深圳北': '京沪高速泰安站方向',
        '厦门北': '京沪高速泰安站方向',
        '曲阜东': '京沪高速泰安站方向',
    }
    stcode = ArrLeaveSt.get(station)
    if stcode != None:
        return stcode
    else:
        print("车次:{0} 车站:{1} 未找到".format(trainNUm, station))
        exit()


def checkin(entrance):
    # 终到车
    if entrance == 0:
        return 0
    else:  # 检票口15A 非终到车
        return int(entrance.strip("检票口AB"))


In [4]:
# 生成车次信息
# '列车编号','类型','最高时速','动拖布置','flag' 列车整体信息

for index, row in trainInfo.iterrows():
    trainNum = row["车次"]
    maxspeed = (species.get(trainNum[0]))[0]  # 有字头的列车
    if maxspeed == None:  # 最高速度未找到
        maxspeed = '120'  # 普客

    marshalling = (species.get(trainNum[0]))[1]
    # trainTC=marshalling.get() #动拖布置
    trainDF.loc[index] = [trainNum, "COMMUTER", maxspeed, marshalling, "X1"]
    trainList.append("{0} {1} {2} {3} {4} : ".format(
        trainNum, "COMMUTER", maxspeed, marshalling, "X1"))
trainDF.head()


,车次,类型,最高时速,动拖布置,flag
0,D1601D1604,COMMUTER,200,LPPL,X1
1,D1603D1606,COMMUTER,200,LPPL,X1
2,D1605D1602,COMMUTER,200,LPPL,X1
3,D1607D1608,COMMUTER,200,LPPL,X1
4,D1611,COMMUTER,200,LPPL,X1


In [5]:
mainstation = "武汉"

# wd = webdriver.Firefox()
# wd.implicitly_wait(10)
# wd.get('https://www.12306.cn/index/view/infos/ticket_check.html')


def getEntance(wd, mainstation, checi):
    # 输入车次并确认
    element1 = wd.find_element(By.XPATH, '//*[@id="ticket_check_trainNum"]')

    element1.clear()
    element1.send_keys(checi+'\n')
    time.sleep(1)
    element1.send_keys(Keys.ENTER)  # 回车键Enter

    # 点击下拉框以获得停站选项
    time.sleep(1)
    wd.find_element(By.XPATH, '//*[@id="ticketEntranceSel"]').click()

    # 选择停站
    time.sleep(1)
    elements = wd.find_elements(By.CSS_SELECTOR, '.model-select-option > li')
    for ele in elements:  # 在所有停站中遍历信息
        if ele.text == station:
            ele.click()  # 点击下拉框对应选项确认
            # 如果查询到
            btn = wd.find_element(By.CSS_SELECTOR, ".btn")
            btn.click()  # 点击查询按钮
            checkin = wd.find_elements(By.CSS_SELECTOR, ".check-numnew")
            time.sleep(0.5)
            entrance = checkin[0].text  # 得到检票口文本信息
            print("车次:{0},检票口:{1}".format(checi, entrance))
            return entrance
    # 没有查询到则为终到车次
    print("车次:{0} ,{1}站无检票口信息".format(checi, station))
    return 0


In [6]:
# 生成停站信息
for index, row in sheet.iterrows():
    arriveSt = getArrLeaSt(row["车次"], row["始发站"])
    leaveSt = getArrLeaSt(row["车次"], row["终到站"])

    # 上下行编号区分车辆进场股道为1,2道,进场及离场股道
    # 靠左行走
    arriveTrack = station.get(arriveSt)[2]
    leaveTrack = station.get(leaveSt)[3]

    # 停站股道,依照检票口选择,没有则填为0，任意股道
    # 使用int()去掉float类型的.0,防止因为格式问题读不出股道
    row = row.fillna(0)
    stopTrack = checkin(row["检票口"])

    at = row["到时"]

    lt = row["开时"]

    # 统一时间格式

    strTime1 = datetime.datetime.strptime(str(at), "%H:%M:%S")
    strTime2 = datetime.datetime.strptime(str(lt), "%H:%M:%S")

    stopTime = (strTime2-strTime1).seconds/60  # 分钟为单位的停站时间

    # 时间处理部分,由到达时刻推出进场时刻
    arrTime1 = strTime1-datetime.timedelta(minutes=(station.get(arriveSt))[4])
    arrTime2 = str(arrTime1.strftime('%Y-%m-%d %H:%M:%S'))[-8:]  # 仅保留时分秒
    arrTime = datetime.datetime.strptime(arrTime2, "%H:%M:%S")  # 转换格式

    # 离场时刻
    leaveTime1 = strTime2+datetime.timedelta(minutes=(station.get(leaveSt))[4])
    leaveTime2 = str(leaveTime1.strftime('%Y-%m-%d %H:%M:%S'))[-8:]
    leaveTime = datetime.datetime.strptime(leaveTime2, "%H:%M:%S")

    # 进场
    arriveStDF.loc[index] = [arriveSt, arriveTrack, arrTime2, 0]
    arriveStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(arriveSt))[0], arriveTrack, arrTime2, 0))
    # 停站
    stopStDF.loc[index] = [ThisStation, stopTrack, row["到时"], stopTime]
    stopStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(ThisStation))[0], stopTrack, row["到时"], int(stopTime)))

    # 离场
    leaveStDF.loc[index] = [leaveSt, leaveTrack, leaveTime2, 0]
    leaveStList.append("{0}#{1}#{2}#{3}".format(
        (station.get(leaveSt))[0], leaveTrack, leaveTime2, 0))


车次:D1611 车站:青岛北 
车次:D1612 车站:青岛北 
车次:D1615 车站:烟台 
车次:D1616 车站:烟台 
车次:D1621 车站:烟台 
车次:D1622 车站:烟台 
车次:D1625 车站:烟台 
车次:D1626 车站:烟台 
车次:D1631 车站:青岛北 
车次:D1632 车站:青岛北 
车次:D1635 车站:青岛 
车次:D1636 车站:青岛 
车次:D1673 车站:青岛北 
车次:D1674 车站:青岛北 
车次:D6004D6053 车站:青岛 
车次:D6004D6053 车站:烟台 
车次:D6006 车站:青岛 
车次:D6007 车站:青岛 
车次:D6015 车站:青岛 
车次:D6017 车站:青岛 
车次:D6023 车站:青岛 
车次:D6024D6023 车站:青岛 
车次:D6024D6023 车站:青岛 
车次:D6051 车站:烟台 
车次:D6055 车站:烟台 
车次:D6061 车站:烟台 
车次:D6063 车站:烟台 
车次:D6077 车站:烟台 
车次:D6078 车站:烟台 
车次:D6080 车站:烟台 
车次:D6087 车站:烟台 
车次:D6089 车站:烟台 
车次:D8175 车站:烟台 
车次:D8176 车站:烟台 
车次:D8181 车站:青岛 
车次:D8183 车站:青岛 
车次:D8184G6921 车站:青岛 
车次:D8184G6921 车站:青岛 
车次:D8185D8186 车站:青岛北 
车次:D8185D8186 车站:青岛北 
车次:G1063 车站:青岛 
车次:G1066 车站:青岛 
车次:G1067 车站:青岛 
车次:G1068 车站:青岛 
车次:G1069 车站:青岛 
车次:G1070 车站:青岛 
车次:G1071 车站:青岛 
车次:G1073 车站:青岛 
车次:G1074 车站:青岛 
车次:G1075 车站:青岛 
车次:G1076 车站:青岛 
车次:G1077 车站:青岛 
车次:G1079 车站:青岛 
车次:G1080 车站:青岛 
车次:G1083 车站:烟台 
车次:G1086 车站:烟台 
车次:G1088 车站:烟台 
车次:G1089 车站:烟台 
车次:G1090 车站:烟台 
车次:G1091

In [7]:
print(arriveStDF.head(), '\n', stopStDF.head(), '\n', leaveStDF.head())  # 部分信息
# wd.quit()


       车站名称 股道      到达时间  停站时间
0  石济客专齐河方向  1  11:07:00     0
1  石济客专齐河方向  1  16:35:00     0
2  石济客专齐河方向  1  12:10:00     0
3  石济客专齐河方向  1  19:14:00     0
4  石济客专齐河方向  1  10:03:00     0 
    车站名称  股道      到达时间  停站时间
0  济南西站   2  11:13:00  28.0
1  济南西站   2  16:41:00  24.0
2  济南西站   2  12:16:00  24.0
3  济南西站   2  19:20:00  29.0
4  济南西站   4  10:09:00   5.0 
         车站名称 股道      到达时间  停站时间
0   石济客专齐河方向  2  11:47:00     0
1   石济客专齐河方向  2  17:11:00     0
2   石济客专齐河方向  2  12:46:00     0
3   石济客专齐河方向  2  19:55:00     0
4  京济联络线济南方向  1  10:22:00     0


**最终车次结果**  
合并车辆信息和停站信息  
  
**导出部分**  
导出为train.txt手动附加原文件头部之后替换原时刻表文件  

In [8]:
timeTable = open(file=TextPath, mode="w")  # 覆盖写入txt
finalRes = []
for i in range(0, len(arriveStList)):
    tstr = "{0}{1} {2} {3} ".format(
        trainList[i], arriveStList[i], stopStList[i], leaveStList[i])
    finalRes.append(tstr)  # 用于展示的列表
    tstr1 = tstr+'\n'
    timeTable.writelines(tstr1)  # 按行写入

timeTable.close()
finalRes


['D1601D1604 COMMUTER 200 LPPL X1 : g#1#11:07:00#0 d#2#11:13:00#28 g#2#11:47:00#0 ',
 'D1603D1606 COMMUTER 200 LPPL X1 : g#1#16:35:00#0 d#2#16:41:00#24 g#2#17:11:00#0 ',
 'D1605D1602 COMMUTER 200 LPPL X1 : g#1#12:10:00#0 d#2#12:16:00#24 g#2#12:46:00#0 ',
 'D1607D1608 COMMUTER 200 LPPL X1 : g#1#19:14:00#0 d#2#19:20:00#29 g#2#19:55:00#0 ',
 'D1611 COMMUTER 200 LPPL X1 : g#1#10:03:00#0 d#4#10:09:00#5 b#1#10:22:00#0 ',
 'D1612 COMMUTER 200 LPPL X1 : b#2#16:12:00#0 d#17#16:20:00#2 g#2#16:28:00#0 ',
 'D1615 COMMUTER 200 LPPL X1 : g#1#16:52:00#0 d#4#16:58:00#4 b#1#17:10:00#0 ',
 'D1616 COMMUTER 200 LPPL X1 : b#2#13:39:00#0 d#16#13:47:00#3 g#2#13:56:00#0 ',
 'D1621 COMMUTER 200 LPPL X1 : g#1#09:35:00#0 d#4#09:41:00#12 b#1#10:01:00#0 ',
 'D1622 COMMUTER 200 LPPL X1 : b#2#20:17:00#0 d#17#20:25:00#4 g#2#20:35:00#0 ',
 'D1625 COMMUTER 200 LPPL X1 : g#1#18:39:00#0 d#1#18:45:00#8 b#1#19:01:00#0 ',
 'D1626 COMMUTER 200 LPPL X1 : b#2#11:22:00#0 d#17#11:30:00#4 g#2#11:40:00#0 ',
 'D1631 COMMUTER 200 LP